Now, we're going to import our cleaned data, and perform some calculations and store it again as processed data for future EDA

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
crude_df = pd.read_csv(r'../data/processed/crude_futures.csv')
gasoil_df = pd.read_csv(r'../data/processed/gasoil_futures.csv')

In [3]:
crude_df["Date"] = pd.to_datetime(crude_df["Date"])
gasoil_df["Date"] = pd.to_datetime(gasoil_df["Date"])

Now we want to compute the crackspread. Note that for this calculation, we need only the close daily prices. 

But before that we need to check that dates are aligned (thus dropping dates not present in both)

We will acheive this using 'merge'. The merging criterion is 'inner' since we want an intersection of dates. And we'd need to drop/only select Date and close of both

In [4]:
crude_df = crude_df[['Date', 'Close']].rename(columns={'Close': 'close_crude', 'Date': 'date'})
gasoil_df = gasoil_df[['Date', 'Close']].rename(columns={'Close': 'close_gasoil', 'Date':'date'})

In [5]:
crack_spread_df = crude_df.merge(gasoil_df, how='inner', on='date')
crack_spread_df

,date,close_crude,close_gasoil
0,2009-05-08,58.14,482.00
1,2009-05-11,57.48,481.50
2,2009-05-12,57.94,483.25
3,2009-05-13,58.12,483.75
4,2009-05-14,58.59,472.75
...,...,...,...
4172,2025-09-02,69.14,703.50
4173,2025-09-03,67.60,707.25
4174,2025-09-04,66.99,698.75
4175,2025-09-05,65.50,676.75


The units of both crude oil and gas oil are NOT the same. They need to be converted to the same $/bbl. Thus we will do that here.

- Brent Crude Futures - \$/bbl
- Low Sulphur Gas Oil - \$/metric tonne

Approximate conversion listed according to BP Approximate conversion factors

In [6]:
#TODO: Change crude and gas oil units
crack_spread_df['close_gasoil'] = crack_spread_df['close_gasoil'] / 7.46
crack_spread_df

,date,close_crude,close_gasoil
0,2009-05-08,58.14,64.611260
1,2009-05-11,57.48,64.544236
2,2009-05-12,57.94,64.778820
3,2009-05-13,58.12,64.845845
4,2009-05-14,58.59,63.371314
...,...,...,...
4172,2025-09-02,69.14,94.302949
4173,2025-09-03,67.60,94.805630
4174,2025-09-04,66.99,93.666220
4175,2025-09-05,65.50,90.717158


Now that we have a merged, cleaned dataframe, let's perform the crack spread calculation and save it

In [7]:
crack_spread_df['crack_spread'] = crack_spread_df['close_gasoil'] - crack_spread_df['close_crude']
crack_spread_df

,date,close_crude,close_gasoil,crack_spread
0,2009-05-08,58.14,64.611260,6.471260
1,2009-05-11,57.48,64.544236,7.064236
2,2009-05-12,57.94,64.778820,6.838820
3,2009-05-13,58.12,64.845845,6.725845
4,2009-05-14,58.59,63.371314,4.781314
...,...,...,...,...
4172,2025-09-02,69.14,94.302949,25.162949
4173,2025-09-03,67.60,94.805630,27.205630
4174,2025-09-04,66.99,93.666220,26.676220
4175,2025-09-05,65.50,90.717158,25.217158


Now save the data

In [8]:
crack_spread_df.to_csv(r'../data/processed/crack_spread.csv')